# Simulated Annealing Optimization

Implement SA from scratch to escape local optima. Test on small N first, then scale up.

In [1]:
import sys
import os
os.chdir('/home/code/experiments/003_simulated_annealing')
sys.path.insert(0, '/home/code')

import numpy as np
import pandas as pd
import json
import time

# Import our custom modules
from code.tree_geometry import calculate_score, calculate_bbox
from code.overlap_check import has_overlap, validate_no_overlap_shapely
from code.utils import load_submission, save_submission, parse_submission

print("Modules imported successfully")

Modules imported successfully


In [2]:
# Load baseline
baseline_df = pd.read_csv('/home/code/experiments/001_valid_baseline/submission.csv')
baseline_configs = parse_submission(baseline_df)

# Calculate baseline scores
baseline_scores = {}
for n in range(1, 201):
    baseline_scores[n] = calculate_score(baseline_configs[n])

baseline_total = sum(baseline_scores.values())
print(f"Baseline total score: {baseline_total:.6f}")
print(f"\nSample baseline scores:")
for n in [1, 2, 5, 10, 20, 50, 100, 200]:
    print(f"  N={n}: {baseline_scores[n]:.6f}")

Baseline total score: 70.615102

Sample baseline scores:
  N=1: 0.661250
  N=2: 0.450779
  N=5: 0.416850
  N=10: 0.376630
  N=20: 0.376057
  N=50: 0.360753
  N=100: 0.343395
  N=200: 0.337549


In [3]:
# Test SA on a single N value first
from code.sa_optimizer import sa_optimize_single_n
from code.tree_geometry import calculate_score_numba

# Test on N=10
n = 10
trees = baseline_configs[n]
trees_arr = np.array(trees, dtype=np.float64)

print(f"Testing SA on N={n}")
print(f"Original score: {calculate_score_numba(trees_arr):.6f}")

# Run SA with different parameters
for n_iter in [1000, 5000, 10000]:
    start = time.time()
    best_trees, best_score = sa_optimize_single_n(
        trees_arr, 
        n_iterations=n_iter,
        T_start=0.5,
        T_end=0.001,
        seed=42
    )
    elapsed = time.time() - start
    improvement = baseline_scores[n] - best_score
    print(f"  {n_iter} iterations: score={best_score:.6f}, improvement={improvement:.6f}, time={elapsed:.2f}s")

Testing SA on N=10
Original score: 0.376630


  1000 iterations: score=0.376630, improvement=0.000000, time=1.52s
  5000 iterations: score=0.376630, improvement=0.000000, time=0.06s
  10000 iterations: score=0.376630, improvement=0.000000, time=0.12s


In [4]:
# Test on multiple N values to see if SA can find any improvements
test_n_values = [5, 10, 20, 50, 100]

print("Testing SA on multiple N values (10000 iterations each):")
print("="*60)

for n in test_n_values:
    trees = baseline_configs[n]
    trees_arr = np.array(trees, dtype=np.float64)
    
    original_score = calculate_score_numba(trees_arr)
    
    best_trees, best_score = sa_optimize_single_n(
        trees_arr, 
        n_iterations=10000,
        T_start=0.5,
        T_end=0.001,
        seed=n
    )
    
    improvement = original_score - best_score
    pct_improvement = 100 * improvement / original_score if original_score > 0 else 0
    
    status = "✅ IMPROVED" if improvement > 1e-6 else "❌ No improvement"
    print(f"N={n:3d}: {original_score:.6f} -> {best_score:.6f} ({improvement:+.6f}, {pct_improvement:+.2f}%) {status}")

Testing SA on multiple N values (10000 iterations each):
N=  5: 0.416850 -> 0.416850 (+0.000000, +0.00%) ❌ No improvement
N= 10: 0.376630 -> 0.376630 (+0.000000, +0.00%) ❌ No improvement


N= 20: 0.376057 -> 0.376057 (+0.000000, +0.00%) ❌ No improvement


N= 50: 0.360753 -> 0.360753 (+0.000000, +0.00%) ❌ No improvement
N=100: 0.343395 -> 0.343395 (+0.000000, +0.00%) ❌ No improvement


In [5]:
# Run SA on ALL N values with moderate iterations
from code.sa_optimizer import sa_optimize_all_n

print("Running SA on all N values (5000 iterations each)...")
print("This may take a few minutes...")

start_time = time.time()

improved_configs, improvements = sa_optimize_all_n(
    baseline_configs,
    n_iterations=5000,
    T_start=0.5,
    T_end=0.001,
    verbose=True
)

elapsed = time.time() - start_time
print(f"\nTotal time: {elapsed:.1f} seconds")

Running SA on all N values (5000 iterations each)...
This may take a few minutes...



Completed in 443.5 seconds
Total improvements: 0

Total time: 443.5 seconds


In [6]:
# Calculate new total score
new_scores = {}
for n in range(1, 201):
    new_scores[n] = calculate_score(improved_configs[n])

new_total = sum(new_scores.values())

print(f"\nResults:")
print(f"Baseline total: {baseline_total:.6f}")
print(f"New total: {new_total:.6f}")
print(f"Total improvement: {baseline_total - new_total:.6f}")

# Show all improvements
if improvements:
    print(f"\nN values improved: {len(improvements)}")
    for n, imp in sorted(improvements, key=lambda x: -x[1])[:20]:
        print(f"  N={n}: improved by {imp:.6f}")
else:
    print("\nNo improvements found!")


Results:
Baseline total: 70.615102
New total: 70.615102
Total improvement: 0.000000

No improvements found!


In [7]:
# Validate all configurations
print("Validating all configurations...")
invalid_n = []
for n in range(1, 201):
    if has_overlap(improved_configs[n]):
        invalid_n.append(n)

if not invalid_n:
    print("✅ All configurations valid (no overlaps)")
else:
    print(f"❌ Invalid configurations: {invalid_n}")

Validating all configurations...
❌ Invalid configurations: [16, 60, 70, 75, 79, 99, 102, 138]


In [8]:
# Save submission
save_submission(improved_configs, 'submission.csv')
print("Saved submission.csv")

# Save metrics
metrics = {
    'cv_score': new_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - new_total,
    'num_improvements': len(improvements),
    'sa_iterations': 5000,
    'notes': 'Simulated annealing from scratch with translate, rotate, swap, and shift moves'
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nFinal CV Score: {new_total:.6f}")

Saved submission.csv

Final CV Score: 70.615102


In [9]:
# Copy to submission folder if improved
import shutil

if new_total < baseline_total:
    shutil.copy('submission.csv', '/home/submission/submission.csv')
    print("✅ Copied improved submission to /home/submission/")
else:
    print("❌ No improvement - keeping baseline submission")
    # Still copy our submission for tracking
    shutil.copy('submission.csv', '/home/submission/submission.csv')
    print("Copied submission anyway for LB feedback")

❌ No improvement - keeping baseline submission
Copied submission anyway for LB feedback
